### Install and import libaries
References (detailed links to functions and examples in each cell):
- Semantic Link Labs library: https://github.com/microsoft/semantic-link-labs
    - Wiki: https://github.com/microsoft/semantic-link-labs/wiki
    - sempy_labs package: https://semantic-link-labs.readthedocs.io/en/latest/sempy_labs.html
    - Code Examples: https://github.com/microsoft/semantic-link-labs/wiki/Code-Examples
    - Helper Notebooks: https://github.com/microsoft/semantic-link-labs/tree/main/notebooks
#
- Semantic Link: https://learn.microsoft.com/en-us/fabric/data-science/semantic-link-overview
    - Sempy Package: https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy?view=semantic-link-python
    - Fabric Functions Code Examples: https://github.com/m-kovalsky/Fabric

This notebook utilizes the Semantic Link Labs library to dynamically deploy Power BI semantic models and reports across various workspaces. This specific use case involves maintaining a single master model and report, which are then duplicated across multiple workspaces and Lakehouses to accommodate diverse data sources (e.g., by store). Key benefits include centralized governance, automated deployments, separate source tables (potentially smaller row counts), and reduced need for Row-Level Security (RLS), thereby enhancing performance.


In [36]:
# Install the library in a Fabric notebook: https://github.com/microsoft/semantic-link-labs?tab=readme-ov-file#install-the-library-in-a-fabric-notebook

%pip install semantic-link-labs

StatementMeta(, 7563fd63-2233-47d3-903a-e91bbcdde292, 55, Finished, Available, Finished)


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [37]:
# Import the libraries: https://github.com/microsoft/semantic-link-labs?tab=readme-ov-file#once-installed-run-this-code-to-import-the-library-into-your-notebook

import sempy.fabric as fabric
import sempy_labs as labs
from sempy_labs import migration, directlake, admin, graph
from sempy_labs import lakehouse as lake
from sempy_labs import report as rep
from sempy_labs.tom import connect_semantic_model
from sempy_labs.report import ReportWrapper

StatementMeta(, 7563fd63-2233-47d3-903a-e91bbcdde292, 57, Finished, Available, Finished)

### Programmatically deploy semantic models and reports with Lakehouse schemas

1. Deploy semantic model to new workspace and rename
2. Update semantic model connection to new Lakehouse
3. (Optional) Check semantic model Lakehouse connection
4. Update Direct Lake table partition to new schema
5. (Optional) Get Tabular Model Scripting Language (TMSL) to confirm lineage
6. Clone report to new workspace and rebind to new semantic model
7. (Optional) Launch report to preview

In [38]:
source_dataset = 'sales_dashhboard_DL' # semantic model
source_dataset_workspace = 'AnalyticsPlatformSA'
source_report = 'sales_dashboard_dl'
source_report_workspace = 'AnalyticsPlatformSA'

StatementMeta(, 7563fd63-2233-47d3-903a-e91bbcdde292, 58, Finished, Available, Finished)

In [39]:
# Initialize lists for each parameter

target_dataset = 'sales_dashboard_DL'
target_workspace = 'AnalyticsPlatformSpoke'
target_lakehouse = 'sales_gold'
target_lakehouse_workspace = 'AnalyticsPlatformSpoke'
target_schema = ''
target_report = 'sales_dashboard_dl'
target_report_workspace = 'AnalyticsPlatformSpoke'



StatementMeta(, 7563fd63-2233-47d3-903a-e91bbcdde292, 59, Finished, Available, Finished)

In [40]:
# Deploy semantic model to new workspace

"""
    Function - deploy_semantic_model: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.deploy_semantic_model 
        Code Example: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.html#sempy_labs.deploy_semantic_model
"""

import sempy_labs as labs

refresh_target_dataset = False
overwrite = True

    
labs.deploy_semantic_model(
        source_dataset = source_dataset,
        source_workspace = source_dataset_workspace,
        target_dataset = target_dataset,
        target_workspace = target_workspace,
        refresh_target_dataset = refresh_target_dataset,
        overwrite = overwrite
    )

StatementMeta(, 7563fd63-2233-47d3-903a-e91bbcdde292, 60, Finished, Available, Finished)

🟢 The 'sales_dashboard_DL' semantic model has been created within the 'AnalyticsPlatformSpoke' workspace.


In [41]:
# Update semantic model connection to new Lakehouse

"""
     Function - update_direct_lake_model_lakehouse_connection: https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.directlake.html#sempy_labs.directlake.update_direct_lake_model_lakehouse_connection
     sempy_labs.directlake.update_direct_lake_model_connection
"""

from sempy_labs.directlake import update_direct_lake_model_connection

update_direct_lake_model_connection(
         dataset = target_dataset,
         workspace =  target_workspace,
         source = target_lakehouse,
         source_type = 'Lakehouse',
         source_workspace = target_lakehouse_workspace,
         use_sql_endpoint  = False
    )

StatementMeta(, 7563fd63-2233-47d3-903a-e91bbcdde292, 61, Finished, Available, Finished)

🟢 The expression in the 'sales_dashboard_DL' semantic model within the 'AnalyticsPlatformSpoke' workspace has been updated to point to the 'sales_gold' lakehouse in the 'AnalyticsPlatformSpoke' workspace.


In [42]:
# And now refresh it

labs.refresh_semantic_model(
    dataset = target_dataset,
    workspace = target_workspace
    )

StatementMeta(, 7563fd63-2233-47d3-903a-e91bbcdde292, 62, Finished, Available, Finished)

⌛ Refresh of the 'sales_dashboard_DL' semantic model within the 'AnalyticsPlatformSpoke' workspace is in progress...
🟢 Refresh 'full' of the 'sales_dashboard_DL' semantic model within the 'AnalyticsPlatformSpoke' workspace is complete.


Note: 
- After initial model deployment: need to update security role members, cloud connection from SSO (default) to fixed identity, access permissions.
- Once deployed and model is overwritten only cloud connection needs to be updated.